In [1]:
import sys
from datasets import load_dataset
import torch
from trl.trainer.dpo_trainer import DPOTrainer

sys.path.append('..')
from model.causal_lm import AutoLMFineTuner
from model.utils import LMHyperParams, SmModel, ModelChoice
from train import start_training
from dataset.squad import UltraFeedbackDataModule
%load_ext autoreload
%autoreload 2

/admin/home-brianf/smolmodels/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-14 04:12:22.870 | INFO     | train:<module>:5 - Loading dependencies - Torch...
2024-11-14 04:12:22.967 | INFO     | train:<module>:16 - Loading dependencies - Lightning...
2024-11-14 04:12:22.967 | INFO     | train:<module>:25 - Loading dependencies - Project...


In [2]:
params = LMHyperParams(base_model_checkpoint="Qwen/Qwen2.5-0.5B", warmup_steps_count=10, tuning_type="dpo", train_batch_size=1)

# https://github.com/philschmid/deep-learning-pytorch-huggingface/blob/main/training/dpo-align-llms-in-2024-with-trl.ipynb

In [3]:
model = AutoLMFineTuner(params)


2024-11-14 04:12:39.274 | INFO     | model.causal_lm:__init__:69 - Using DPO with LoraConfig: LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type='CAUSAL_LM', inference_mode=False, r=256, target_modules='all-linear', lora_alpha=128, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False))


In [4]:
data_module = UltraFeedbackDataModule(16, model.tokenizer, 1024)
data_module.cpu_count = 12
data_module.setup("fit")

2024-11-14 04:12:42.686 | INFO     | dataset.squad:setup:214 - Loading dataset for stage fit


2024-11-14 04:12:44.654 | INFO     | dataset.squad:setup:222 - Processing dataset for stage fit, workers: 12, cache dir dataset_caches/ultrafeedback


In [5]:
start_training(
    "16-mixed", "", model, model.tokenizer, data_module, params, False, ""
)


Seed set to 42


2024-11-14 04:12:47.076 | INFO     | train:start_training:268 - Effective batch size: 2 (2x1)
/admin/home-brianf/smolmodels/.venv/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /admin/home-brianf/smolmodels/.venv/lib/python3.11/s ...
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA H100 80GB HBM3') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
2024-11-14 04:12:47.122 | INFO 

Configuring optimizers: 0



  | Name            | Type                 | Params | Mode 
-----------------------------------------------------------------
0 | model           | PeftModelForCausalLM | 634 M  | train
1 | reference_model | Qwen2ForCausalLM     | 494 M  | eval 
-----------------------------------------------------------------
140 M     Trainable params
988 M     Non-trainable params
1.1 B     Total params
4,515.347 Total estimated model params size (MB)
1682      Modules in train mode
686       Modules in eval mode


Epoch 0:   0%|          | 0/3770 [00:00<?, ?it/s]                          

OutOfMemoryError: CUDA out of memory. Tried to allocate 9.27 GiB. GPU 0 has a total capacity of 79.11 GiB of which 8.06 GiB is free. Including non-PyTorch memory, this process has 71.04 GiB memory in use. Of the allocated memory 63.11 GiB is allocated by PyTorch, and 7.27 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)